In [ ]:
# ================================================
# 📌 Power BI Python Prediction Script
# - Predicts Dynamic Today +6 Days Data Everyday
# ================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta

# ================================================
# 1️⃣ Upload historical data
# ================================================
train_df = pd.read_csv("C:\Users\ADMIN\Documents\Weather Analysis Project\Historical Data(2010-2025-07-31)CSV\bangalore_weather_2010_2025-07-31_final.csv")

# ================================================
# 2️⃣ Feature engineering
# ================================================
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['dayofyear'] = train_df['date'].dt.dayofyear
train_df['year'] = train_df['date'].dt.year

features = ['dayofyear', 'year']

xgb_targets = [
    "temperature_2m_min",
    "temperature_2m_max",
    "temperature_2m_mean"
]

rf_targets = [
    "windspeed_10m_max",
    "windgusts_10m_max",
    "shortwave_radiation_sum",
    "precipitation_sum",
    "cloudcover_mean",
    "humidity_2m_mean"
]

today = datetime.today().date()
predict_dates = pd.date_range(start=today, end=today + timedelta(days=6), freq='D')
predict_df = pd.DataFrame({'date': predict_dates})
predict_df['dayofyear'] = predict_df['date'].dt.dayofyear
predict_df['year'] = predict_df['date'].dt.year

predictions = {'date': predict_df['date']}

# ================================================
# 3️⃣ RandomForest regression targets
# ================================================
for target in xgb_targets:
    X_train = train_df[features]
    y_train = train_df[target]

    model = RandomForestRegressor(n_estimators=500, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(predict_df[features])
    predictions[target] = preds
# ================================================
# 4️⃣ Random Forest regression targets
# ================================================
rf_features = features + xgb_targets
for target in rf_targets:
    X_train_rf = train_df[rf_features]
    y_train_rf = train_df[target]

    train_valid = X_train_rf.copy()
    train_valid['y'] = y_train_rf
    train_valid = train_valid.dropna()

    if len(train_valid) == 0:
        continue

    X_train_rf = train_valid[rf_features]
    y_train_rf = train_valid['y']

    X_future_rf = predict_df.copy()
    for t in xgb_targets:
        X_future_rf[t] = predictions[t]
    X_future_rf = X_future_rf[rf_features]

    model_rf = RandomForestRegressor(n_estimators=200, random_state=42)
    model_rf.fit(X_train_rf, y_train_rf)

    preds_rf = model_rf.predict(X_future_rf)
    predictions[target] = preds_rf

# ================================================
# 5️⃣ WeatherSimple classifier
# ================================================
def map_weather_simple(code):
    if code in [0, 1, 2, 3]:
        return 0
    elif code in [51, 53, 55]:
        return 1
    elif code in [61, 63, 65]:
        return 2
    else:
        return 2

train_df['WeatherSimple'] = train_df['weathercode'].apply(map_weather_simple)

clf_features = features + [
    "temperature_2m_max",
    "windgusts_10m_max",
    "shortwave_radiation_sum"
]

# Add only if available
if 'cloudcover_mean' in predictions:
    clf_features.append("cloudcover_mean")
if 'humidity_2m_mean' in predictions:
    clf_features.append("humidity_2m_mean")

X_cls = train_df[clf_features]
y_cls = train_df['WeatherSimple']

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

clf = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
clf.fit(X_train_cls, y_train_cls)

# Predict future
for t in xgb_targets + rf_targets:
    if t in predictions:
        predict_df[t] = predictions[t]

y_future_pred = clf.predict(predict_df[clf_features])
predictions['WeatherSimple'] = y_future_pred

# ================================================
# 6️⃣ Return final Dataframe for Power BI
# ================================================
final_columns = [
    'date',
    'temperature_2m_min',
    'temperature_2m_max',
    'temperature_2m_mean',
    'windspeed_10m_max',
    'windgusts_10m_max',
    'shortwave_radiation_sum',
    'precipitation_sum',
    'WeatherSimple'
]

if 'cloudcover_mean' in predictions:
    final_columns.insert(-1, 'cloudcover_mean')
if 'humidity_2m_mean' in predictions:
    final_columns.insert(-1, 'humidity_2m_mean')

predicted_df = pd.DataFrame({col: predictions[col] for col in final_columns})
predicted_df